In [14]:
import pandas as pd
import src.nasty_score as nasty

### Read data and load models

In [15]:
data = pd.read_feather('data/nasty_data_2021.fea').fillna(-9999)
pitchers = pd.read_feather('data/pitcher_ids.fea')

In [16]:
sz_model = nasty.import_model('models/sz_model.pkl')
swing_model = nasty.import_model('models/swing_model.pkl')
swingmiss_model = nasty.import_model('models/swingmiss_model.pkl')

### Assign nasty scores

In [17]:
data = nasty.add_nasty_score(data, sz_model, swing_model, swingmiss_model)
data = data.merge(pitchers, on='pitcher', how='left')

### Tables

#### Table 1. Top 10 nastiest individual pitches of 2021

In [18]:
table_cols = ['player_name', 'game_date', 'pitch_type', 'release_speed',
              'release_spin_rate', 'nasty_score']

data[table_cols].sort_values(by='nasty_score', ascending=False).head(10)

,player_name,game_date,pitch_type,release_speed,release_spin_rate,nasty_score
437495,"Treinen, Blake",2021-06-14,SL,86.9,2415,100.000000
418316,"Rodón, Carlos",2021-06-18,SL,88.1,2558,99.345627
418250,"Rodón, Carlos",2021-06-18,SL,89.0,2612,97.871368
164865,"Rodón, Carlos",2021-08-26,SL,85.1,2430,97.773918
418270,"Rodón, Carlos",2021-06-18,SL,89.2,2448,97.706215
5923,"Treinen, Blake",2021-10-12,SL,87.4,2498,97.092041
282097,"Treinen, Blake",2021-07-27,SL,87.1,2199,96.784599
351015,"Rodón, Carlos",2021-07-06,SL,90.0,2492,95.361206
441757,"Rodón, Carlos",2021-06-13,SL,90.0,2657,95.202896
625214,"Matzek, Tyler",2021-04-26,SL,87.9,2840,94.931953


#### Table 2. Top 10 nastiest pitches of 2021

In [19]:
ranked_pitches = data.groupby(['player_name', 'pitch_type']).agg(
    avg_release_speed=('release_speed', 'mean'),
    avg_release_spin_rate=('release_spin_rate', 'mean'),
    pitch_count = pd.NamedAgg('nasty_score', 'count'),
    nasty_score = pd.NamedAgg('nasty_score', 'mean')).reset_index()

ranked_pitches.loc[ranked_pitches.pitch_count > 150]\
    .sort_values(by='nasty_score', ascending=False).head(10)

,player_name,pitch_type,avg_release_speed,avg_release_spin_rate,pitch_count,nasty_score
3218,"Treinen, Blake",SL,86.508625,2396.773893,429,45.877842
1056,"Gallegos, Giovanny",SL,85.600375,2482.917603,534,43.852406
2977,"Snell, Blake",SL,86.527448,2394.909091,572,43.711437
2689,"Rodón, Carlos",SL,85.703834,2431.174121,626,43.335186
1062,"Gallen, Zac",SL,86.148603,2467.815642,179,43.237019
1528,"Jackson, Luke",SL,87.618769,2313.284615,650,42.098763
2473,"Pineda, Michael",SL,81.226923,1890.672065,494,41.436024
1666,"Kittredge, Andrew",SL,88.922958,2720.390728,453,41.283775
553,"Chafin, Andrew",SL,82.4624,2407.472,250,41.260658
959,"Feyereisen, J.P.",CH,87.619403,1476.114428,201,40.953033


#### Table 3. Top 10 nastiest pitchers of 2021

In [20]:
ranked_pitches = data.groupby(['player_name']).agg(
    nasty_score = pd.NamedAgg('nasty_score', 'mean'),
    pitch_count = pd.NamedAgg('nasty_score', 'count')).reset_index()

ranked_pitches.loc[ranked_pitches.pitch_count > 400]\
    .sort_values(by='nasty_score', ascending=False).head(10)

,player_name,nasty_score,pitch_count
906,"deGrom, Jacob",34.452480,1226
874,"Williams, Devin",31.847212,993
354,"Hendriks, Liam",31.606447,1172
420,"Kimbrel, Craig",30.885818,1055
389,"Jackson, Luke",30.825329,1204
332,"Hader, Josh",30.705772,995
434,"Kopech, Michael",29.817446,1225
733,"Scott, Tanner",29.626652,1032
300,"Glasnow, Tyler",28.324326,1339
603,"Pagán, Emilio",28.170403,1098


#### Table 4. Top 10 nastiest games of 2021

In [21]:
ranked_pitches = data.groupby(['player_name', 'game_date']).agg(
    nasty_score = pd.NamedAgg('nasty_score', 'mean'),
    pitch_count = pd.NamedAgg('nasty_score', 'count')).reset_index()

ranked_pitches.loc[ranked_pitches.pitch_count > 50]\
    .sort_values(by='nasty_score', ascending=False).head(10)

,player_name,game_date,nasty_score,pitch_count
21860,"deGrom, Jacob",2021-06-11,40.249886,80
21858,"deGrom, Jacob",2021-05-31,39.085972,70
21861,"deGrom, Jacob",2021-06-16,38.241863,51
21854,"deGrom, Jacob",2021-04-23,38.216568,109
21851,"deGrom, Jacob",2021-04-05,36.835072,77
10670,"Kopech, Michael",2021-10-03,36.208549,51
21857,"deGrom, Jacob",2021-05-25,35.273743,63
21852,"deGrom, Jacob",2021-04-10,35.157982,95
21853,"deGrom, Jacob",2021-04-17,34.179119,99
21855,"deGrom, Jacob",2021-04-28,33.255737,93
